```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Implicit Equal Weights Particle Filter

This notebook implements prototyping and example/demo of the Implicit Equal Weights Particle Filter (IEWPF).


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import pyopencl
import os
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BaseOceanStateEnsemble
from SWESimulators import DataAssimilationUtils as DAUtils


In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
cl_queue = pyopencl.CommandQueue(cl_ctx)
print "Using ", cl_ctx.devices[0].name

# Ensemble

We need an ensemble where each particle
- runs an independent ocean model
- drift a drifter
- applies a localized small-scale error
- observes the drifter position

Needs to be done:
- Initialize models (create netcdf with init, add error with amp 10*q0(?), put drifter into a small area of the 
- make useful plots to evaluate the results
    - Suggestion: 3-line [eta, hu, hv] plot, with truth, ensemble (mean field with individual drifters), mean-square diff?
    - 3x3/4x4/5x5 plot of eta from different ensemble members?
    - Standard animation of a single ensemble member.


## Create initial condition for ensemble:

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 10.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
if True:
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5


if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)
print "q0: ", q0
print "[f, g, H]", [f, g, waterDepth]
print "f/gH: ", f/(g*waterDepth)
print "gH/f: ", g*waterDepth/f

reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
sim = CDKLM16.CDKLM16(cl_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)

ensemble_size = 5
ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(ensemble_size, cl_ctx)
ensemble.setGridInfoFromSim(sim)
ensemble.setStochasticVariables(initialization_variance_factor=0, observation_variance_factor=2,
                                small_scale_perturbation_amplitude=q0)
ensemble.init()


oldSchoolAnimation = True
if not oldSchoolAnimation:
    T = 300
    sub_t = 10*dt
    for i in range(T):
        t = ensemble.step(sub_t)
        if i == T-1:
            ensemble.printMaxOceanStates()
            
            
    kskugasfd

fig = plt.figure()
plotter = PlotHelper.EnsembleAnimator(fig, ensemble, trueStateOnly=True)

T = 11
sub_t = 300*dt
def animate(i):
    if (i>0):
        t = ensemble.step(sub_t)
    else:
        t = 0.0

    plotter.plot(ensemble);

    fig.suptitle("Ensemble = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) 

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
ensemble.plotEnsemble()

In [ ]:
max_dt = ensemble.findLargestPossibleTimeStep()
print "Largest possible timestep with this case: ", max_dt

In [ ]:
ensemble.plotDistanceInfo()

# Implementing IEWPF

The IEWPF algorithm for the observation at time $t^m$ is applyed to the model at time $t^{m-1}$.
The algorithm consists of the following steps:
1. Find target weight using only deterministic evolution of the model from $t^{m-1}$ to $t^m$.
- Draw a sample from $\xi \sim N(0,P)$.
- Move particles towards observation using a applying some scaling of $\xi$

Some challenges:
- As long as we have one and only one drifter, constant $H(x,y)$ and double periodic boundary conditions, the linear problem within step 1 has the same matrix for any observation. This matrix should therefore be pre-calculated.
- During the creation of $\xi$ we need the random numbers, so using functionality of the OceanNoiseClass would be nice.


Questions:
- Should the IEWPF subroutines/functions/methods be under its own class? Or under the OceanNoiseClass

## Step 0
Pure deterministic integration of the model.

In [ ]:
t = ensemble.step(dt, apply_stochastic_term=False)

## Step 1

The full covariance matrix can be written as
$$ Q = Q^{1/2} Q^{1/2,T} = U_D U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T U_D^T $$
so that 
$$ HQH^T = H U_D U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T U_D^T  H^T$$

For Step 1 we first need to find $S := (H Q H^T + R)^-1$, which we also will need for later.
We therefore store S, before finding the target weight.

In [ ]:
debug = False

def showMatrices(x, y, title, z = None):
    num_cols = 2
    if z is not None:
        num_cols = 3
    fig = plt.figure(figsize=(num_cols*2,2))
    plt.subplot(1,num_cols,1)
    plt.imshow(x.copy(), origin="lower", interpolation="None")
    plt.xlabel('(%.2E, %.2E)' % (np.min(x), np.max(x)))
    plt.subplot(1,num_cols,2)
    plt.imshow(y.copy(), origin="lower", interpolation="None")
    plt.xlabel('(%.2E, %.2E)' % (np.min(y), np.max(y)))
    if z is not None:
        plt.subplot(1, num_cols, 3)
        plt.imshow(z.copy(), origin="lower", interpolation="None")
        plt.xlabel('(%.2E, %.2E)' % (np.min(z), np.max(z)))
    plt.suptitle(title)
    
# Copied from RandomNumberGenerator.ipynb  
def SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, q0, L):
    dist = np.sqrt( dx*dx*(a_x - b_x)**2  +  dy*dy*(a_y - b_y)**2)
    return q0*(1.0 + dist/L)*np.exp(-dist/L)


def createS(ensemble, const_H):
    """
    Create the 2x2 matrix S = (HQH^T + R)^-1
    
    Constant as long as
     - one drifter only,
     - H(x,y) = const, and
     - double periodic boundary conditions
    """
    
    dt = ensemble.dt
    dx = ensemble.dx
    dy = ensemble.dy
    geoBalanceConst = ensemble.g*const_H/(2.0*ensemble.f)
    
    # These should be read from a OceanStateNoise object?
    q0 = ensemble.small_scale_perturbation_amplitude
    L = 0.75*dx 
    
    # Local storage for x and y correlations:
    x_corr = np.zeros((7,7))
    y_corr = np.zeros((7,7))
    tmp_x = np.zeros((7,7))
    tmp_y = np.zeros((7,7))
    
    # Mid_coordinates:
    mid_i, mid_j = 3, 3
    
    # Fill the buffers with U_{GB}^T U_D^T  H^T
    x_corr[mid_j+1, mid_i] = -geoBalanceConst*dt/dy
    x_corr[mid_j-1, mid_i] =  geoBalanceConst*dt/dy
    y_corr[mid_j, mid_i+1] =  geoBalanceConst*dt/dx
    y_corr[mid_j, mid_i-1] = -geoBalanceConst*dt/dx
    if debug: showMatrices(x_corr, y_corr, "$U_{GB}^T U_D^T  H^T$")
    
    # Apply the SOAR function to fill x and y with 7x5 and 5x7 respectively
    # First for x:
    for j,i in [mid_j+1, mid_i], [mid_j-1, mid_i]:
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                tmp_x[b, a] += x_corr[j,i]*SOAR_Q(a, b, i, j, dx, dy, q0, L)
    # Then for y:
    for j,i in [mid_j, mid_i+1], [mid_j, mid_i-1]:
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                #print SOAR_Q(a, b, i, j, dx, dy, q0, L)
                tmp_y[b, a] += y_corr[j,i]*SOAR_Q(a, b, i, j, dx, dy, q0, L)
    if debug: showMatrices(tmp_x, tmp_y, "$Q_{SOAR} U_{GB}^T U_D^T  H^T$")      
            
    # Apply the SOARfunction again to fill the points needed to find drift in (mid_i, mid_j)
    # For both x and y:
    for j,i in [mid_j+1, mid_i], [mid_j-1, mid_i], [mid_j, mid_i-1], [mid_j, mid_i+1]:
        x_corr[j,i] = 0
        y_corr[j,i] = 0
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                SOAR_Q_res = SOAR_Q(a, b, i, j, dx, dy, q0, L)
                x_corr[j,i] += tmp_x[b, a]*SOAR_Q_res
                y_corr[j,i] += tmp_y[b, a]*SOAR_Q_res
        if debug: print x_corr[j,i], y_corr[j,i]
    if debug: showMatrices(x_corr, y_corr, "$Q_{SOAR} Q_{SOAR} U_{GB}^T U_D^T  H^T$")
    
    # geostrophic balance:
    x_hu = -geoBalanceConst*(x_corr[mid_j+1, mid_i  ] - x_corr[mid_j-1, mid_i  ])/dy
    x_hv =  geoBalanceConst*(x_corr[mid_j  , mid_i+1] - x_corr[mid_j  , mid_i-1])/dx
    y_hu = -geoBalanceConst*(y_corr[mid_j+1, mid_i  ] - y_corr[mid_j-1, mid_i  ])/dy
    y_hv =  geoBalanceConst*(y_corr[mid_j  , mid_i+1] - y_corr[mid_j  , mid_i-1])/dx 
    
    # Drift
    HQHT = np.matrix([[x_hu*dt, y_hu*dt],[x_hv*dt, y_hv*dt]])    
    if debug: print HQHT
    S_inv = HQHT + ensemble.observation_cov
    if debug: print S_inv
    S = np.linalg.inv(S_inv)
    if debug: print S
    return S
    
S = createS(ensemble, 10.0)
print "S: ", S


We expected $HQH^T$ to be a full $2 \times 2$ matrix, but we see now that the $x$ and $y$ coordinate of the drifter is uncorrelated.

When one thinks about it, this makes sense. Even though we know the $x$ position of a drifter, we would have no knowledge of the $y$ position of it. 

The same argument can be used about the velocities. Can we really say anything about the $hu$ in a cell based on the value of $hv$ in the same cell? From $hu(x_i, y_j)$ alone, we can not say whether all the momentum is in $x$-direction, or if it is just a part of the total momentum, which also has a $y$ component. Even though the geostrophic balance generate rotating velocity fields, we will have no knowledge on where in the rotation we are. 

In [ ]:
# As we have S = (HQH^T + R)^-1, we can do step 1 of the IEWPF algorithm
def obtainTargetWeight(ensemble, S):
    d = ensemble.getInnovations()
    Ne = ensemble.getNumParticles()
    c = np.zeros(Ne)
    for i in range(Ne):
        e = np.dot(S, d[i,:])
        c[i] = 0.5*np.dot(e,d[i,:])
        print "c[" + str(i) + "]: ", c[i]
        print "exp(-c[" + str(i) + "]: ", np.exp(-c[i])
    return np.min(c)

target_weight = obtainTargetWeight(ensemble, S)
print "target_weight: ", target_weight
print "current weights: "
print DAUtils.getGaussianWeight(ensemble.getDistances(), \
                                ensemble.getObservationVariance())

## Step 2
Draw samples from $\xi \sim N(0, P)$, where 
$$ P = (Q^{-1}+ H^T R^{-1} H)^{-1}. $$
With some additional linear algebra magic, we can rewrite $P$ as
$$ P = Q - QH^T (HQH^T + R)^{-1} H Q \\ = Q - Q H^T S H Q \\ = Q^{1/2}(I - Q^{1/2}H^T S H Q^{1/2}) Q^{1/2}$$
By using the three-step covariance structure, we get
$$ P = U_D U_{GB} \tilde{Q}^{1/2} \left[ I - U_D U_{GB} \tilde{Q}^{1/2} H^T S H  \tilde{Q}^{1/2}U_{GB}^T U_D^T \right] \tilde{Q}^{1/2}U_{GB}^T U_D^T $$

In order to produce $\xi$, we therefore draw $\tilde{\xi} \sim N(0, I)$, where $\tilde{\xi} \in \mathbb{R}^{N_x}$, and find $\xi = P \tilde{\xi}$.
To do this matrix multiplication (or really, to apply this operator), we start by applying $\tilde{Q}^{1/2}U_{GB}^T U_D^T $ to $\tilde{\xi}$.

#### Note
Note that P depends on the position of the observed drifter!

In [ ]:
if 'noise' in globals():
    noise.cleanUp()
    
noise = OceanStateNoise.OceanStateNoise.fromsim(ensemble.particles[0], soar_q0=q0)

def drawFromP(noise, S, drifter_pos, sim, const_H, debug=False):
    nx, ny = noise.nx, noise.ny
    
    # 1) Allocate memory and prepare stuff
    # 1.1) P depends on the position of the drifter, and we therefore need to know
    #      in which cell it is.
    
    # Based on periodic boundary conditions on CDKLM:
    x_zero_ref = 2
    y_zero_ref = 2
    
    cell_id_x = int(np.ceil(drifter_pos[0]/sim.dx) + x_zero_ref)
    cell_id_y = int(np.ceil(drifter_pos[1]/sim.dy) + y_zero_ref)
    
    # 1.2) Allocate data
    p_eta = np.zeros((ny, nx))
    p_hu = np.zeros((ny+2, nx+2))
    p_hv = np.zeros((ny+2, nx+2))
    
    # 2) Generate xi by filling p_eta, p_hu and p_hv with random numbers 
    noise.generateUniformDistributionCPU()
    p_eta = noise.getRandomNumbersCPU().copy()
    noise.generateUniformDistributionCPU()
    p_hu[1:-1, 1:-1] = noise.getRandomNumbersCPU().copy()
    noise.generateUniformDistributionCPU()
    p_hv[1:-1, 1:-1] = noise.getRandomNumbersCPU().copy()
    
    # 3) Apply \tilde{Q}^{1/2}U_{GB}^T U_D^T to 
    # 3.1) U_D^T
    # 3.1.1) Generate random numbers for drifter
    u_x, noise.host_seed[cell_id_y, cell_id_x/2] = noise._lcg(noise.host_seed[cell_id_y, cell_id_x/2])
    u_y, noise.host_seed[cell_id_y, cell_id_x/2] = noise._lcg(noise.host_seed[cell_id_y, cell_id_x/2])
    
    # 3.1.2) Add them to the velocity field
    p_hu[cell_id_y+1, cell_id_x+1] += sim.dt*u_x
    p_hv[cell_id_y+1, cell_id_x+1] += sim.dt*u_y
    if debug: showMatrices(p_eta, p_hu, "uniform random numbers with $U_D xi$", p_hv)
    
    # 3.2) U_{GB}^T
    #      Apply an inverse geostrophic balance from hu and hv onto eta
    #      !! Need halo of 1 cell for hu and hv
    # 3.2.1) Enforce periodic boundary conditions
    p_hu[0,:] = p_hu[-2,:]
    p_hu[-1,:] = p_hu[1,:]
    p_hv[:,0] = p_hv[:,-2]
    p_hv[:,-1] = p_hv[:,1]
    
    # 3.2.2) Compute new values for eta
    #        Assuming const H
    geo_balance_const = sim.g*const_H/sim.f
    if debug: print "geo_balance_const", geo_balance_const
    for j in range(ny):
        y = j+1
        for i in range(nx):
            x = i+1 
            p_eta[j, i] -= 0.5*geo_balance_const*(-(p_hu[y+1,x] - p_hu[y-1,x])/sim.dy + (p_hv[y,x+1] - p_hv[y,x-1])/sim.dx)
            
    if debug: showMatrices(p_eta, p_hu, "uniform random numbers with $U_{GB}^T U_D^T xi$", p_hv)
    
    # 3.3) Q^{1/2}
    #      We fill the noise.random_numbers_host buffer with p_eta, and use noise._applyQ_CPU()
    noise.random_numbers_host = p_eta
    p_eta = noise._applyQ_CPU()[1:-1, 1:-1] 
    if debug: showMatrices(p_eta, p_hu, "uniform random numbers with $Q^{1/2} U_{GB}^T U_D^T xi$", p_hv)
    
    # 4) Generate the local term Q^{1/2}^T H^T S H U Q^{1/2}
    # 4.1) Make a new local copy of the area needed for this calculation, reading periodic boundaries
    local_eta = np.zeros((7,7))
    for j in range(7):
        j_global = (cell_id_y-3+j+sim.ny)%sim.ny 
        for i in range(7):
            i_global = (cell_id_x-3+i+sim.nx)%sim.nx 
            local_eta[j,i] = p_eta[j_global, i_global]
    if debug: showMatrices(local_eta, p_eta, "local $\eta$ from global $\eta$")
    
    # 4.2) Q^{1/2}: Apply SOAR to find the laplace-stencil points
    # Apply the SOARfunction again to fill the points needed to find drift in (mid_i, mid_j)
    # For both x and y:
    local_eta_soar = np.zeros(4) # representing [north, east, south, west] positions from 
    north_east_south_west_index = [[4,3,0], [3,4,1], [2,3,2], [3,2,3]] #[[y-index-eta, x-index-eta, index-local_eta_soar]]
    for j,i,soar_res_index in north_east_south_west_index:
        if debug: print (j,i), soar_res_index
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                local_eta_soar[soar_res_index] += local_eta[b,a]*SOAR_Q(a, b, i, j, sim.dx, sim.dy, noise.soar_q0, noise.soar_L)
    if debug: print local_eta_soar
    
    # 4.3) U: Apply geostrophic balance and drift to local_huv
    local_pos = [0, 0]
    local_pos[0] = -sim.dt*geo_balance_const*(local_eta_soar[0] - local_eta_soar[2])/(2*sim.dy)
    local_pos[1] =  sim.dt*geo_balance_const*(local_eta_soar[1] - local_eta_soar[3])/(2*sim.dx)
    if debug: print local_pos
    
    # 4.4) H: The observation operator discards then all but these two local_drift terms:
    #      H * [zeros_eta, zeros_hu, zeros_hv, local_pos_x, local_pos_y] = [local_pos_x, local_pos_y]
    
    # 4.4) Multiply with S
    local_pos = np.dot(S, local_pos)
    if debug: print local_pos
    
    # 4.5) H^T: The transpose of the observation operator now maps the drifter position to the complete state vector:
    #      H^T [local_pos_x, local_pos_y] = [zeros_eta, zeros_hu, zeros_hv, local_pos_x, local_pos_y]
    
    # 4.6) U^T: multiply with dt (drift) and map out to laplacian stencil
    local_eta_soar[0] = -local_pos[0,1]*geo_balance_const*sim.dt/(2*sim.dy) # north
    local_eta_soar[2] =  local_pos[0,1]*geo_balance_const*sim.dt/(2*sim.dy) # south
    local_eta_soar[1] =  local_pos[0,0]*geo_balance_const*sim.dt/(2*sim.dx) # east
    local_eta_soar[3] = -local_pos[0,0]*geo_balance_const*sim.dt/(2*sim.dx) # west
    
    # 4.7) Q^{1/2}:
    local_eta.fill(0.0)
    for j,i,soar_res_index in north_east_south_west_index:
        if debug: print (j,i), soar_res_index
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                local_eta[b,a] += local_eta_soar[soar_res_index]*SOAR_Q(a, b, i, j, sim.dx, sim.dy, 
                                                                        noise.soar_q0, noise.soar_L)
    if debug: showMatrices(local_eta, p_eta, "local $\eta$ from global $\eta$")
    
    # 5) xi - local_term_from-4):
    old_p_eta = p_eta.copy()
    for j in range(7):
        j_global = (cell_id_y-3+j+sim.ny)%sim.ny 
        for i in range(7):
            i_global = (cell_id_x-3+i+sim.nx)%sim.nx 
            p_eta[j_global, i_global] -= local_eta[j,i]
            #p_eta[j_global, i_global] -= 5000000000*local_eta[j,i]
    if debug: showMatrices(old_p_eta, p_eta, "local $\eta$ from global $\eta$")
    
    # 6) Apply SOAR, Geo-balance and drift to make this a sample from N(0,P)
    # 6.1) U_{GB} Q^{1/2}: Apply SOAR and geostrophic balance
    old_p_eta = p_eta.copy()
    noise.random_numbers_host = p_eta
    p_eta, p_hu, p_hv = noise._obtainOceanPerturbations_CPU(ensemble.base_H, sim.f, sim.coriolis_beta, sim.g)
    if debug: showMatrices(old_p_eta, p_eta, "last $Q^{1/2}$")
    if debug: showMatrices(p_eta, p_hu, "sample from $N(0,P)$", p_hv)
    
    # 6.2) Drift term
    p_x = sim.dt*p_hu[cell_id_y, cell_id_x]/waterDepth
    p_y = sim.dt*p_hv[cell_id_y, cell_id_x]/waterDepth
    
    return p_eta[1:-1, 1:-1], p_hu, p_hv, p_x, p_y
    
observed_particles = ensemble.observeParticles()
#print observed_particles

xi = [None]*ensemble.getNumParticles()

#for p in range(ensemble.getNumParticles()):
for p in range(1):
    p_eta, p_hu, p_hv, p_x, p_y = drawFromP(noise, S, observed_particles[p, :], \
                                            ensemble.particles[p], waterDepth, debug=True)
    xi[p] = [p_eta, p_hu, p_hv, p_x, p_y]
print "Done"

In [ ]:
for p in range(ensemble.getNumParticles()):
    showMatrices(xi[p][0], xi[p][1], "particle " + str(p), xi[p][2])
    print "Got (p_x, p_y): ", (xi[p][3], xi[p][4])

## Step 3

We now use the samples of $\xi$ to push each particle towards the observation.

In [ ]:
def pushParticleTowardsObservation(sim, noise, S, \
                                   observation, innovation, xi, const_H, target_weight, debug=True):
    # Following the 3rd step of the IEWPF algorithm
    
    # 0) Define constants/buffers etc
    geo_balance_const = sim.g*const_H/sim.f
    Nx = ensemble.getNumParticles()*1.0
    
    # Based on periodic boundary conditions on CDKLM:
    x_zero_ref = 2
    y_zero_ref = 2
    
    cell_id_x = int(np.ceil(observation[0]/sim.dx) + x_zero_ref)
    cell_id_y = int(np.ceil(observation[1]/sim.dy) + y_zero_ref)
    
    # 1) Solve linear problem
    e = np.dot(S, innovation)
    if debug: print "e: ", e
    
    # 2) K = QH^T e = U_D U_GB Q^{1/2} Q^{1/2} U_GB^T U_D^T  H^T e
    #    Obtain the Kalman gain
    # 2.1) U_GB^T U_D^T H^T e
    # 2.1.1) H^T: The transpose of the observation operator now maps the drifter position to the complete state vector:
    #        H^T [e_x, e_y] = [zeros_eta, zeros_hu, zeros_hv, e_x, e_y]
    
    # 2.1.2) U_GB^T U_D^T: multiply with dt (drift) and map out to laplacian stencil
    local_huhv = np.zeros(4) # representing [north, east, south, west] positions from 
    north_east_south_west_index = [[4,3,0], [3,4,1], [2,3,2], [3,2,3]] #[[y-index-eta, x-index-eta, index-local_eta_soar]]
    local_huhv[0] = -e[0,1]*geo_balance_const*sim.dt/(2*sim.dy) # north
    local_huhv[2] =  e[0,1]*geo_balance_const*sim.dt/(2*sim.dy) # south
    local_huhv[1] =  e[0,0]*geo_balance_const*sim.dt/(2*sim.dx) # east
    local_huhv[3] = -e[0,0]*geo_balance_const*sim.dt/(2*sim.dx) # west
    
    # 2.1.3) Q^{1/2}:
    local_eta = np.zeros((7,7))
    for j,i,soar_res_index in north_east_south_west_index:
        if debug: print (j,i), soar_res_index
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                local_eta[b,a] += local_huhv[soar_res_index]*SOAR_Q(a, b, i, j, sim.dx, sim.dy, 
                                                                    noise.soar_q0, noise.soar_L)
    if debug: showMatrices(local_eta, local_eta, "local $\eta$ from global $\eta$")
    
    # 2.2) Apply U_D U_GB Q^{1/2} to the result
    # 2.2.1)  Easiest way: map local_eta to a global K_eta_tmp buffer
    K_eta_tmp = np.zeros((ny, nx))
    for j in range(7):
        j_global = (cell_id_y-3+j+sim.ny)%sim.ny 
        for i in range(7):
            i_global = (cell_id_x-3+i+sim.nx)%sim.nx 
            K_eta_tmp[j_global, i_global] += local_eta[j,i]
            #K_eta_tmp[j_global, i_global] += 10000000*local_eta[j,i]
    if debug: showMatrices(K_eta_tmp, local_eta, "global K_eta from local K_eta, halfway in the calc.")
        
    # 2.2.2) Use K_eta_tmp as the noise.random_numbers_host
    noise.random_numbers_host = K_eta_tmp
    
    # 2.2.3) Apply soar + geo-balance
    K_eta , K_hu, K_hv = noise._obtainOceanPerturbations_CPU(ensemble.base_H, sim.f, sim.coriolis_beta, sim.g)
    K_eta = K_eta[1:-1, 1:-1]
    if debug: showMatrices(K_eta, K_hu, "Kalman gain", K_hv)
    
    # 2.2.4) Find drift:
    K_x = sim.dt*K_hu[cell_id_y, cell_id_x]/waterDepth
    K_y = sim.dt*K_hv[cell_id_y, cell_id_x]/waterDepth
    if debug: print "Kx, Ky: ", (K_x, K_y)
    
    # 3) Obtain phi = d^T * e
    phi = innovation[0]*e[0,0] + innovation[1]*e[0,1]
    if debug: print "phi: ", phi
    
    # 4) Obtain x_a
    nudged_eta, nudged_hu, nudged_hv = sim.download()
    nudged_eta = nudged_eta[2:-2, 2:-2] + K_eta
    nudged_hu = nudged_hu[2:-2, 2:-2] + K_hu
    nudged_hv = nudged_hv[2:-2, 2:-2] + K_hv
    if debug: print "Shapes of nudged vars: ", nudged_eta.shape, nudged_hu.shape, nudged_hv.shape
    if debug: showMatrices(nudged_eta, nudged_hu, "$x_a$", nudged_hv)
    
    nudged_x = observation[0] + K_x
    nudged_y = observation[1] + K_y
    if debug: print "nudged_x, nudged_y: ", (nudged_x, nudged_y)

    # 5) obtain gamma
    if debug: print "Shapes of xi: ", xi[0].shape, xi[1].shape, xi[2].shape
    if debug: print "xi_x, xi_y: ", (xi[3], xi[4])
    gamma = xi[3]*xi[3] + xi[4]*xi[4]
    for field in range(3):
        for j in range(ny):
            for i in range(nx):
                gamma += xi[field][j,i]*xi[field][j,i]
    if debug: print "gamma: ", gamma
    
    # 6) Find a
    a = phi + target_weight
    if debug: print "a = gamma + target_weight: ", a
        
    # 7) Solving the Lambert W function
    #alpha = 10000
    lambert_W_arg = -(gamma/Nx)*np.exp(a/Nx)*np.exp(-gamma/Nx)
    alpha = -(Nx/gamma)*np.real(lambertw(lambert_W_arg, k=-1))
    
    if debug: print "Check a against the Lambert W requirement: ", a, " < ", - Nx + gamma - Nx*np.log(gamma/Nx), " = ", a <  - Nx + gamma - Nx*np.log(gamma/Nx)
    if debug: print "-e^-1 < z < 0 : ", -1.0/np.exp(1), " < ", lambert_W_arg, " < ", 0, " = ", \
        (-1.0/np.exp(1) < lambert_W_arg, lambert_W_arg < 0)
    if debug: print "Obtained alpha: ", alpha
    if debug: print "The two branches from Lambert W: ", (lambertw(lambert_W_arg), lambertw(lambert_W_arg, k=-1))
    print "The two branches from Lambert W: ", (np.real(lambertw(lambert_W_arg)), np.real(lambertw(lambert_W_arg, k=-1)))
    
    # 7.2) alpha*xi
    showMatrices(alpha*xi[0], alpha*xi[1], "alpha * xi", alpha*xi[2])
    
    # 8) Final nudge!
    nudged_eta += alpha*xi[0]
    nudged_hu += alpha*xi[1]
    nudged_hv += alpha*xi[2]
    nudged_x += alpha*xi[3]
    nudged_y += alpha*xi[4]
    showMatrices(nudged_eta, nudged_hu, "Nudged field", nudged_hv)
    print "Original pos (x, y): ", (observation[0], observation[1])
    print "Final nudged_x, nudged_y: ", (nudged_x, nudged_y)
    print "Diff original - nudged: ", (observation[0] - nudged_x, observation[1] - nudged_y)
        
    return None
        
innovations = ensemble.getInnovations()
for p in range(1): #ensemble.getNumParticles()):
    pushParticleTowardsObservation(ensemble.particles[p], noise, S, \
                                   observed_particles[p, :], innovations[p], xi[p], waterDepth, 
                                   target_weight, True)

In [ ]:
print (37 + 8) % 40 
print (2 - 2 + 40) % 40

print S
print np.dot(S, [1,2])

something = np.ones((7,7))
print something
something.fill(0)
print something
print '%.2E, %.2E,' % (np.min(S), np.max(S))

print ensemble.observation_cov